# Libraries

In [57]:
import pandas as pd
import numpy as np
import os

# Loading Data

In [58]:
raw_data_path = "C:/Users/Saisa/Downloads/Walmart_Sales/data/raw/Walmart_Sales.csv"
store_data_path = "C:/Users/Saisa/Downloads/Walmart_Sales/data/raw/store_names.csv"

df_sales = pd.read_csv(raw_data_path)
df_stores = pd.read_csv(store_data_path)

In [59]:
df_sales.shape

(6435, 8)

In [60]:
df_sales.head()

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106


In [61]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6435 entries, 0 to 6434
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         6435 non-null   int64  
 1   Date          6435 non-null   object 
 2   Weekly_Sales  6435 non-null   float64
 3   Holiday_Flag  6435 non-null   int64  
 4   Temperature   6435 non-null   float64
 5   Fuel_Price    6435 non-null   float64
 6   CPI           6435 non-null   float64
 7   Unemployment  6435 non-null   float64
dtypes: float64(5), int64(2), object(1)
memory usage: 402.3+ KB


# Data Cleaning

In [62]:
# Standardzing the Column Names
df_sales.columns = df_sales.columns.str.lower().str.replace(" ", "_")
df_stores.columns = df_stores.columns.str.lower().str.replace(" ", "_")

In [63]:
#Parsing Date Columns
df_sales['date'] = pd.to_datetime(df_sales['date'], errors='coerce')
df_sales.dropna(subset=['date'], inplace=True)  # remove rows with invalid dates


In [64]:
# Handling Missing Values
df_sales.dropna()
df_sales.isna().sum()


store           0
date            0
weekly_sales    0
holiday_flag    0
temperature     0
fuel_price      0
cpi             0
unemployment    0
dtype: int64

In [65]:
# Removing Duplicate Values

df_sales.drop_duplicates(inplace=True)


In [66]:
# Extracting Time Components

df_sales['year'] = df_sales['date'].dt.year
df_sales['month'] = df_sales['date'].dt.month
df_sales['day'] = df_sales['date'].dt.day
df_sales['weekday'] = df_sales['date'].dt.dayofweek
df_sales['is_weekend'] = df_sales['weekday'].isin([5, 6])

In [68]:
# Step 1: Import holidays
import holidays

# Step 2: Define US holidays for all years present in the dataset
us_holidays = holidays.US(years=range(df_sales['date'].dt.year.min(), df_sales['date'].dt.year.max() + 1))

# Step 3: Create 'is_holiday' flag
df_sales['is_holiday'] = df_sales['date'].isin(us_holidays).astype(int)

# Step 4: Create 'holiday_name' column
df_sales['holiday_name'] = df_sales['date'].map(us_holidays)

# Step 5: Fill non-holiday rows with 'None'
df_sales['holiday_name'] = df_sales['holiday_name'].fillna('No Holiday')


C:\Users\saisa\AppData\Local\Temp\ipykernel_7936\3313458946.py:8: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df_sales['is_holiday'] = df_sales['date'].isin(us_holidays).astype(int)


In [69]:
# Merge Store Names
df = pd.merge(df_sales, df_stores, how='left', on='store')
df['store_name'].isna().sum()


np.int64(0)

In [70]:
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2565 entries, 0 to 2564
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   store         2565 non-null   int64         
 1   date          2565 non-null   datetime64[ns]
 2   weekly_sales  2565 non-null   float64       
 3   holiday_flag  2565 non-null   int64         
 4   temperature   2565 non-null   float64       
 5   fuel_price    2565 non-null   float64       
 6   cpi           2565 non-null   float64       
 7   unemployment  2565 non-null   float64       
 8   year          2565 non-null   int32         
 9   month         2565 non-null   int32         
 10  day           2565 non-null   int32         
 11  weekday       2565 non-null   int32         
 12  is_weekend    2565 non-null   bool          
 13  is_holiday    2565 non-null   int64         
 14  holiday_name  2565 non-null   object        
 15  store_name    2565 non-null   object  

In [71]:
processed_path = "C:/Users/Saisa/Downloads/Walmart_Sales/data/processed/processed_sales.csv"
df.to_csv(processed_path, index=False)
